## De - Para

**Geral**
- **spend** - Gasto
- **cpm** - CPM
- **ctr** - CTR
- **reach** - Alcance
- **impressions** - Impressões

**Descoberta**
- **post_reaction** - Reações
- **post_engagement** - Engajamentos com a publicação
- **video_p25_watched_actions** - 25% Video View
- **video_p50_watched_actions** - 50% Video View
- **video_p75_watched_actions** - 75% Video View
- **page_engagement** - Engajamento com a página


**Conversão**
- **offsite_conversion.fb_pixel_add_to_cart** - add_to_cart
- **offsite_conversion.fb_pixel_purchase** - Compra
- **purchase** - Compra
- **cost_per_unique_click** - CPC
- **omni_initiated_checkout** - Initiate Checkout
- **frequency** - Frequência
- **clicks** - Cliques

### Imports

In [1]:
from graph_api import *

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import copy
import time

import seaborn as sns
import matplotlib.pyplot as plt

### Conectando API

In [2]:
fb_api = open("tokens/fb_token").read()
ad_acc = "3120164588217844"

graph = GraphAPI(ad_acc, fb_api)

## Resumo de funções

### 1. Leitura de dados

#### 1.1. Campanhas

In [ ]:
# Leitura de dados de campanhas
df_campaigns = graph.get_campaigns_status(date_preset='last_7d', increment=1, process=1).reset_index(drop=True)

# Alternativa de leitura, usando o parâmetro since_until
# df_campaigns_m = graph.get_campaigns_status(since_until=['2022-08-01', datetime.today().strftime("%Y-%m-%d")])

In [ ]:
df_campaigns.head()

In [ ]:
len(df_campaigns)

In [ ]:
# Filtrando para campanhas ativas e que tiveram alguma veiculação no período
df_camp_act = df_campaigns[df_campaigns["status"] == "ACTIVE"].reset_index(drop=True)
df_camp = df_campaigns[df_campaigns['insights'].notna()].reset_index(drop=True)

In [ ]:
df_camp_act.head()

In [ ]:
len(df_camp_act)

In [ ]:
df_camp.head()

In [ ]:
len(df_camp)

#### 1.2. Adsets

In [ ]:
# Leitura de dados de Adsets
df_adsets = graph.get_adset_status(date_preset='last_7d', increment=1, process=1).reset_index(drop=True) # since_until=['2022-08-15', '2022-08-30']

df_adsets_act = df_adsets[df_adsets["status"] == "ACTIVE"].reset_index(drop=True)
df_adsets_data = df_adsets[df_adsets['insights'].notna()].reset_index(drop=True)

In [ ]:
df_adsets.head()

In [ ]:
len(df_adsets)

In [ ]:
df_adsets_act.head()

In [ ]:
len(df_adsets_act)

In [ ]:
df_adsets_data.head()

In [ ]:
len(df_adsets_data)

#### 1.3. Ads

In [ ]:
df_ads = graph.get_ads_status(since_until=[datetime.today().strftime("%Y-%m-%d"),datetime.today().strftime("%Y-%m-%d")], increment=1, process=1).reset_index(drop=True)

In [ ]:
df_ads_act = df_ads[df_ads["status"] == "ACTIVE"].reset_index(drop=True)
df_ads_data = df_ads[df_ads['insights'].notna()].reset_index(drop=True)

In [ ]:
df_ads.head()

In [ ]:
len(df_ads)

In [ ]:
df_ads_act.head()

In [ ]:
len(df_ads_act)

In [ ]:
df_ads_data.head()

In [ ]:
len(df_ads_data)

## 2. Reports

In [ ]:
products = ["ml", "dip","aut","black22"]

In [ ]:
# Carrega estatísticas de campanhas agrupadas
df_ = graph.campaigns_report(df_camp, products, stats=['spend', 'ctr', 'purchase'], kinds=["CONVERSÃO", "ESCALA", "REMARKETING", "CAPTAÇAO"]) #<- Adicionar pivot
# df_.pivot_table(index=["product", "kind", "campaign"])
df_

In [ ]:
graph.adsets_report(df_adsets_data, df_camp, products, stats=['ctr', 'spend'], campaigns='CONVERSÃO', adsets_with='')

## 3. Insights over time

In [ ]:
# Também é possível fazer a leitura dos dados por período, sem organizar informações
graph.process_insights(df_camp, 'spend', products=['ML'], kinds=["CONVERSÃO"])

In [ ]:
# Também é possível fazer a leitura dos dados por período, sem organizar informações
graph.process_insights(df_adsets_data, 'purchase', products=['ML'], kinds=["REMARKETING"])

## 4. Análise Exploratória

### 4.1. Quantidade por produto e tipo

In [ ]:
# Quantidade de campanhas por produto e tipo
df_camp.groupby(['product','kind']).agg({'name':'count'}).sort_values(by=['product','name'], ascending=False)

In [ ]:
plt.figure(figsize=(12,6))
plt.title("Quantidade de campanhas por produto e tipo")

ax = sns.countplot(data=df_camp, x='product', hue='kind')

In [ ]:
# Quantidade de asdsets por produto e tipo
df_adsets_data.groupby(['product','kind']).agg({'name':'count'}).sort_values(by=['product','name'], ascending=False)

In [ ]:
plt.figure(figsize=(12,6))
plt.title("Quantidade de adsets por produto e tipo")

ax = sns.countplot(data=df_adsets_data, x='product', hue='kind')

In [ ]:
# Quantidade de asds por produto e tipo
df_ads_data.groupby(['product','kind']).agg({'name':'count'}).sort_values(by=['product','name'], ascending=False)

In [ ]:
plt.figure(figsize=(12,6))
plt.title("Quantidade de ads por produto e tipo")

ax = sns.countplot(data=df_ads_data, x='product', hue='kind')

### 4.2. KPI's por campanha e tipo

In [ ]:
# KPI's por campanha e tipo
df_camp_stats = graph.campaigns_report(df_camp, stats=['spend','cpc','cpm','ctr','reach','purchase_roas'], products=['DIP','ML','AUT','BLACK22'])
df_camp_stats

### 4.3. KPI's sobre o tempo por campanha e tipo

#### 4.3.0. Agregando os dados

In [ ]:
df_camp['insights'][1]['data'][1]['purchase_roas'][0]['value']

In [ ]:
df_camp_insights = pd.DataFrame()

In [ ]:
df2 = pd.DataFrame({'name':None}, index=[0])

#df2 = pd.DataFrame({'name':[None],'product':[None],'type':[None],'date_start':[None],'date_stop':[None],'spend':[None]})
for i in range(len(df_camp)):
    for j in range(len(df_camp['insights'][i]['data'])):
        df2['name'] = df_camp['name'][i]
        df2['status'] = df_camp['status'][i]
        df2['product'] = df_camp['product'][i]
        df2['kind'] = df_camp['kind'][i]
        df2['adsets_amount'] = len(df_camp['adsets'][i]['data'])
        df2['date_start'] = pd.to_datetime(df_camp['insights'][i]['data'][j]['date_start'])
        df2['date_stop'] = pd.to_datetime(df_camp['insights'][i]['data'][j]['date_stop'])
        df2['spend'] = float(df_camp['insights'][i]['data'][j]['spend'])
        df2['clicks'] = int(df_camp['insights'][i]['data'][j]['clicks'])
        df2['impressions'] = int(df_camp['insights'][i]['data'][j]['impressions'])
        df2['ctr'] = df_camp['insights'][i]['data'][j].get('ctr') if df_camp['insights'][i]['data'][j].get('ctr') is None else float(df_camp['insights'][i]['data'][j].get('ctr'))
        df2['cpc'] = df_camp['insights'][i]['data'][j].get('cpc') if df_camp['insights'][i]['data'][j].get('cpc') is None else float(df_camp['insights'][i]['data'][j].get('cpc'))
        df2['cpm'] = df_camp['insights'][i]['data'][j].get('cpm') if df_camp['insights'][i]['data'][j].get('cpm') is None else float(df_camp['insights'][i]['data'][j].get('cpm'))
        df2['video_25'] = df_camp['insights'][i]['data'][j].get('video_p25_watched_actions') if df_camp['insights'][i]['data'][j].get('video_p25_watched_actions') is None else int(df_camp['insights'][i]['data'][j]['video_p25_watched_actions'][0]['value'])
        df2['video_50'] = df_camp['insights'][i]['data'][j].get('video_p50_watched_actions') if df_camp['insights'][i]['data'][j].get('video_p50_watched_actions') is None else int(df_camp['insights'][i]['data'][j]['video_p50_watched_actions'][0]['value'])
        df2['video_75'] = df_camp['insights'][i]['data'][j].get('video_p75_watched_actions') if df_camp['insights'][i]['data'][j].get('video_p75_watched_actions') is None else int(df_camp['insights'][i]['data'][j]['video_p75_watched_actions'][0]['value'])
        
        try:
            x = df_camp['insights'][i]['data'][j]['actions'].index({'action_type': 'purchase', 'value': '1'})
            df2['purchase'] = int(df_camp['insights'][i]['data'][j]['actions'][x]['value'])
            df2['purchase_roas'] = float(df_camp['insights'][i]['data'][j]['purchase_roas'][0]['value'])
        except:
            df2['purchase'] = 0
            df2['purchase_roas'] = 0
        
        df_camp_insights = df_camp_insights.append(df2)
        
df_camp_insights.reset_index(drop=True, inplace=True)

In [ ]:
#dados acumulados por campanha
df_camp_insights['purchase_acc'] = df_camp_insights.groupby('name')['purchase'].cumsum()
df_camp_insights['spend_acc'] = df_camp_insights.groupby('name')['spend'].cumsum()
df_camp_insights['cpa'] = df_camp_insights['spend'] / df_camp_insights['purchase']
df_camp_insights['cpa_acc'] = df_camp_insights['spend_acc'] / df_camp_insights['purchase_acc']

for i in range(len(df_camp_insights)):
    if df_camp_insights['purchase'][i] != 0:
        df_camp_insights['cpa'][i] = df_camp_insights['spend'][i] / df_camp_insights['purchase'][i]
    else:
        df_camp_insights['cpa'][i] = df_camp_insights['spend'][i]
        
for i in range(len(df_camp_insights)):
    if df_camp_insights['purchase_acc'][i] != 0:
        df_camp_insights['cpa_acc'][i] = df_camp_insights['spend_acc'][i] / df_camp_insights['purchase_acc'][i]
    else:
        df_camp_insights['cpa_acc'][i] = df_camp_insights['spend_acc'][i]

df_camp_insights['purchase_value'] = df_camp_insights['purchase_roas'] * df_camp_insights['spend']
df_camp_insights['purchase_value_acc'] = df_camp_insights.groupby('name')['purchase_value'].cumsum()
df_camp_insights['purchase_roas_acc'] = df_camp_insights['purchase_value_acc'] / df_camp_insights['spend_acc']
                
df_camp_insights['clicks_acc'] = df_camp_insights.groupby('name')['clicks'].cumsum()
df_camp_insights['impressions_acc'] = df_camp_insights.groupby('name')['impressions'].cumsum()
df_camp_insights['cpc_acc'] = df_camp_insights['spend_acc'] / df_camp_insights['clicks_acc']
df_camp_insights['cpm_acc'] = (df_camp_insights['spend_acc'] / df_camp_insights['impressions_acc']) * 1000
df_camp_insights['ctr_acc'] = df_camp_insights['clicks_acc'] / df_camp_insights['impressions_acc'] * 100 #em %

df_camp_insights.replace([np.inf, -np.inf], 0, inplace=True)

df_camp_insights['video_25_acc'] = df_camp_insights.groupby('name')['video_25'].cumsum()
df_camp_insights['video_50_acc'] = df_camp_insights.groupby('name')['video_50'].cumsum()
df_camp_insights['video_75_acc'] = df_camp_insights.groupby('name')['video_75'].cumsum()

In [ ]:
pd.options.display.max_columns = 99999
df_camp_insights.head()

In [ ]:
#filtros
product = 'DIP'
kind = 'REMARKETING'
status = 'ACTIVE'

df_filter = df_camp_insights[(df_camp_insights['product']==product) & (df_camp_insights['kind']==kind) & (df_camp_insights['status']==status)]

#### 4.3.1. Spend

In [ ]:
plt.figure(figsize=(12,6))
plt.title("Gasto x Tempo por campanha")

a = sns.lineplot(x='date_start', y='spend', data=df_filter, ci=None, hue='name')

In [ ]:
#spend acumulado
plt.figure(figsize=(12,6))
plt.title("Gasto acumulado x Tempo por campanha")

a = sns.lineplot(x='date_start', y='spend_acc', data=df_filter, ci=None, hue='name')

#### 4.3.2. CPC

In [ ]:
plt.figure(figsize=(12,6))
plt.title("CPC x Tempo por campanha")

a = sns.lineplot(x='date_start', y='cpc', data=df_filter, ci=None, hue='name')

In [ ]:
#cpc acumulado
plt.figure(figsize=(12,6))
plt.title("CPC acumulado x Tempo por campanha")

a = sns.lineplot(x='date_start', y='cpc_acc', data=df_filter, ci=None, hue='name')

#### 4.3.3. CPM

In [ ]:
plt.figure(figsize=(12,6))
plt.title("CPM x Tempo por campanha")

a = sns.lineplot(x='date_start', y='cpm', data=df_filter, ci=None, hue='name')

In [ ]:
#(spend / impressions) * 1000
#cpm acumulado
plt.figure(figsize=(12,6))
plt.title("CPM acumulado x Tempo por campanha")

a = sns.lineplot(x='date_start', y='cpm_acc', data=df_filter, ci=None, hue='name')

#### 4.3.4. CTR

In [ ]:
plt.figure(figsize=(12,6))
plt.title("CTR x Tempo por campanha")

a = sns.lineplot(x='date_start', y='ctr', data=df_filter, ci=None, hue='name')

In [ ]:
#clicks / impressions
#ctr acumulado
plt.figure(figsize=(12,6))
plt.title("CTR acumulado x Tempo por campanha")

a = sns.lineplot(x='date_start', y='ctr_acc', data=df_filter, ci=None, hue='name')

#### 4.3.5. CPA

In [ ]:
plt.figure(figsize=(12,6))
plt.title("CPA x Tempo por campanha")

a = sns.lineplot(x='date_start', y='cpa', data=df_filter, ci=None, hue='name')

In [ ]:
#cpa acumulado
plt.figure(figsize=(12,6))
plt.title("CPA acumulado x Tempo por campanha")

a = sns.lineplot(x='date_start', y='cpa_acc', data=df_filter, ci=None, hue='name')

#### 4.3.6. ROAS

In [ ]:
plt.figure(figsize=(12,6))
plt.title("ROAS x Tempo por campanha")

a = sns.lineplot(x='date_start', y='purchase_roas', data=df_filter, ci=None, hue='name')

In [ ]:
#roas acumulado
plt.figure(figsize=(12,6))
plt.title("ROAS acumulado x Tempo por campanha")

a = sns.lineplot(x='date_start', y='purchase_roas_acc', data=df_filter, ci=None, hue='name')

#### 4.3.7. Video watched

In [ ]:
plt.figure(figsize=(12,6))
plt.title("Video-25% x Tempo por campanha")

a = sns.lineplot(x='date_start', y='video_25', data=df_filter, ci=None, hue='name')

In [ ]:
#video-25 acumulado
plt.figure(figsize=(12,6))
plt.title("Video-25% acumulado x Tempo por campanha")

a = sns.lineplot(x='date_start', y='video_25_acc', data=df_filter, ci=None, hue='name')

In [ ]:
plt.figure(figsize=(12,6))
plt.title("Video-50% x Tempo por campanha")

a = sns.lineplot(x='date_start', y='video_50', data=df_filter, ci=None, hue='name')

In [ ]:
#video-50 acumulado
plt.figure(figsize=(12,6))
plt.title("Video-50% acumulado x Tempo por campanha")

a = sns.lineplot(x='date_start', y='video_50_acc', data=df_filter, ci=None, hue='name')

In [ ]:
plt.figure(figsize=(12,6))
plt.title("Video-75% x Tempo por campanha")

a = sns.lineplot(x='date_start', y='video_75', data=df_filter, ci=None, hue='name')

In [ ]:
#video-75 acumulado
plt.figure(figsize=(12,6))
plt.title("Video-75% acumulado x Tempo por campanha")

a = sns.lineplot(x='date_start', y='video_75_acc', data=df_filter, ci=None, hue='name')

#### Indicadores por quantidade de anúncios

In [ ]:
plt.figure(figsize=(12,6))
plt.title("CPC acumulado x Tempo por qtde de anúncio")

a = sns.lineplot(x='date_start', y='cpc_acc', data=df_camp_insights, ci=None, hue='adsets_amount', style='adsets_amount')

In [ ]:
plt.figure(figsize=(12,6))
plt.title("CPM acumulado x Tempo por qtde de anúncio")

a = sns.lineplot(x='date_start', y='cpm_acc', data=df_camp_insights, ci=None, hue='adsets_amount', style='adsets_amount')

In [ ]:
plt.figure(figsize=(12,6))
plt.title("CTR acumulado x Tempo por qtde de anúncio")

a = sns.lineplot(x='date_start', y='ctr_acc', data=df_camp_insights, ci=None, hue='adsets_amount', style='adsets_amount')

#### Correlacionando indicadores

In [ ]:
df_camp_insights['spend_norm'] = df_camp_insights['spend'] / np.amax(df_camp_insights['spend'], axis=0)
df_camp_insights['cpc_norm'] = df_camp_insights['cpc'] / np.amax(df_camp_insights['cpc'], axis=0)
df_camp_insights['cpm_norm'] = df_camp_insights['cpm'] / np.amax(df_camp_insights['cpm'], axis=0)
df_camp_insights['ctr_norm'] = df_camp_insights['ctr'] / np.amax(df_camp_insights['ctr'], axis=0)

In [ ]:
plt.figure(figsize=(12,6))
plt.title("KPI's x Tempo por campanha")

#a = sns.lineplot(x='date_start', y='spend_norm', data=df_filter, ci=None)
a = sns.lineplot(x='date_start', y='cpc_norm', data=df_filter, ci=None)
a = sns.lineplot(x='date_start', y='cpm_norm', data=df_filter, ci=None)
a = sns.lineplot(x='date_start', y='ctr_norm', data=df_filter, ci=None)

plt.legend(['cpc', 'cpm', 'ctr'])

### 4.4. KPI's sobre o tempo por conjunto de anúncio

#### 4.4.0. Agregando os dados

In [ ]:
df_adsets_data.head()

In [ ]:
len(df_adsets_data['ads'][0]['data'])

In [ ]:
df_adsets_insights = pd.DataFrame()

In [ ]:
df3 = pd.DataFrame({'name':None}, index=[0])

#df2 = pd.DataFrame({'name':[None],'product':[None],'type':[None],'date_start':[None],'date_stop':[None],'spend':[None]})
for i in range(len(df_adsets_data)):
    for j in range(len(df_adsets_data['insights'][i]['data'])):
        df3['name'] = df_adsets_data['name'][i]
        df3['status'] = df_adsets_data['status'][i]
        df3['product'] = df_adsets_data['product'][i]
        df3['kind'] = df_adsets_data['kind'][i]
        df3['ads_amount'] = len(df_adsets_data['ads'][i]['data'])
        df3['date_start'] = pd.to_datetime(df_adsets_data['insights'][i]['data'][j]['date_start'])
        df3['date_stop'] = pd.to_datetime(df_adsets_data['insights'][i]['data'][j]['date_stop'])
        df3['spend'] = float(df_adsets_data['insights'][i]['data'][j]['spend'])
        df3['clicks'] = int(df_adsets_data['insights'][i]['data'][j]['clicks'])
        df3['impressions'] = int(df_adsets_data['insights'][i]['data'][j]['impressions'])
        df3['ctr'] = df_adsets_data['insights'][i]['data'][j].get('ctr') if df_adsets_data['insights'][i]['data'][j].get('ctr') is None else float(df_adsets_data['insights'][i]['data'][j].get('ctr'))
        df3['cpc'] = df_adsets_data['insights'][i]['data'][j].get('cpc') if df_adsets_data['insights'][i]['data'][j].get('cpc') is None else float(df_adsets_data['insights'][i]['data'][j].get('cpc'))
        df3['cpm'] = df_adsets_data['insights'][i]['data'][j].get('cpm') if df_adsets_data['insights'][i]['data'][j].get('cpm') is None else float(df_adsets_data['insights'][i]['data'][j].get('cpm'))
        df3['video_25'] = df_adsets_data['insights'][i]['data'][j].get('video_p25_watched_actions') if df_adsets_data['insights'][i]['data'][j].get('video_p25_watched_actions') is None else int(df_adsets_data['insights'][i]['data'][j]['video_p25_watched_actions'][0]['value'])
        df3['video_50'] = df_adsets_data['insights'][i]['data'][j].get('video_p50_watched_actions') if df_adsets_data['insights'][i]['data'][j].get('video_p50_watched_actions') is None else int(df_adsets_data['insights'][i]['data'][j]['video_p50_watched_actions'][0]['value'])
        df3['video_75'] = df_adsets_data['insights'][i]['data'][j].get('video_p75_watched_actions') if df_adsets_data['insights'][i]['data'][j].get('video_p75_watched_actions') is None else int(df_adsets_data['insights'][i]['data'][j]['video_p75_watched_actions'][0]['value'])
        
        try:
            x = df_adsets_data['insights'][i]['data'][j]['actions'].index({'action_type': 'purchase', 'value': '1'})
            df3['purchase'] = int(df_adsets_data['insights'][i]['data'][j]['actions'][x]['value'])
            df3['purchase_roas'] = float(df_adsets_data['insights'][i]['data'][j]['purchase_roas'][0]['value'])
        except:
            df3['purchase'] = 0
            df3['purchase_roas'] = 0
        
        df_adsets_insights = df_adsets_insights.append(df3)
        
df_adsets_insights.reset_index(drop=True, inplace=True)

In [ ]:
#dados acumulados por campanha
df_adsets_insights['purchase_acc'] = df_adsets_insights.groupby('name')['purchase'].cumsum()
df_adsets_insights['spend_acc'] = df_adsets_insights.groupby('name')['spend'].cumsum()
df_adsets_insights['cpa'] = df_adsets_insights['spend'] / df_adsets_insights['purchase']
df_adsets_insights['cpa_acc'] = df_adsets_insights['spend_acc'] / df_adsets_insights['purchase_acc']

for i in range(len(df_adsets_insights)):
    if df_adsets_insights['purchase'][i] != 0:
        df_adsets_insights['cpa'][i] = df_adsets_insights['spend'][i] / df_adsets_insights['purchase'][i]
    else:
        df_adsets_insights['cpa'][i] = df_adsets_insights['spend'][i]
        
for i in range(len(df_adsets_insights)):
    if df_adsets_insights['purchase_acc'][i] != 0:
        df_adsets_insights['cpa_acc'][i] = df_adsets_insights['spend_acc'][i] / df_adsets_insights['purchase_acc'][i]
    else:
        df_adsets_insights['cpa_acc'][i] = df_adsets_insights['spend_acc'][i]
        
df_adsets_insights['purchase_value'] = df_adsets_insights['purchase_roas'] * df_adsets_insights['spend']
df_adsets_insights['purchase_value_acc'] = df_adsets_insights.groupby('name')['purchase_value'].cumsum()
df_adsets_insights['purchase_roas_acc'] = df_adsets_insights['purchase_value_acc'] / df_adsets_insights['spend_acc']

df_adsets_insights['clicks_acc'] = df_adsets_insights.groupby('name')['clicks'].cumsum()
df_adsets_insights['impressions_acc'] = df_adsets_insights.groupby('name')['impressions'].cumsum()
df_adsets_insights['cpc_acc'] = df_adsets_insights['spend_acc'] / df_adsets_insights['clicks_acc']
df_adsets_insights['cpm_acc'] = (df_adsets_insights['spend_acc'] / df_adsets_insights['impressions_acc']) * 1000
df_adsets_insights['ctr_acc'] = df_adsets_insights['clicks_acc'] / df_adsets_insights['impressions_acc'] * 100 #em %

df_adsets_insights.replace([np.inf, -np.inf], 0, inplace=True)

df_adsets_insights['video_25_acc'] = df_adsets_insights.groupby('name')['video_25'].cumsum()
df_adsets_insights['video_50_acc'] = df_adsets_insights.groupby('name')['video_50'].cumsum()
df_adsets_insights['video_75_acc'] = df_adsets_insights.groupby('name')['video_75'].cumsum()

In [ ]:
df_adsets_insights.head()

In [ ]:
#filtros
product = 'DIP'
kind = 'REMARKETING'
status = 'ACTIVE'

df_filter_adsets = df_adsets_insights[(df_adsets_insights['product']==product) & (df_adsets_insights['kind']==kind) & (df_adsets_insights['status']==status)]

#### 4.4.1. Spend

In [ ]:
plt.figure(figsize=(12,6))
plt.title("Gasto x Tempo por conjunto de anúncio")

a = sns.lineplot(x='date_start', y='spend', data=df_filter_adsets, ci=None, hue='name')

In [ ]:
#spend acumulado
plt.figure(figsize=(12,6))
plt.title("Gasto acumulado x Tempo por conjunto de anúncio")

a = sns.lineplot(x='date_start', y='spend_acc', data=df_filter_adsets, ci=None, hue='name')

#### 4.4.2. CPC

In [ ]:
plt.figure(figsize=(12,6))
plt.title("CPC x Tempo por conjunto de anúncio")

a = sns.lineplot(x='date_start', y='cpc', data=df_filter_adsets, ci=None, hue='name')

In [ ]:
#cpc acumulado
plt.figure(figsize=(12,6))
plt.title("CPC acumulado x Tempo por conjunto de anúncio")

a = sns.lineplot(x='date_start', y='cpc_acc', data=df_filter_adsets, ci=None, hue='name')

#### 4.4.3. CPM

In [ ]:
plt.figure(figsize=(12,6))
plt.title("CPM x Tempo por conjunto de anúncio")

a = sns.lineplot(x='date_start', y='cpm', data=df_filter_adsets, ci=None, hue='name')

In [ ]:
#cpm acumulado
plt.figure(figsize=(12,6))
plt.title("CPM acumulado x Tempo por conjunto de anúncio")

a = sns.lineplot(x='date_start', y='cpm_acc', data=df_filter_adsets, ci=None, hue='name')

#### 4.4.4. CTR

In [ ]:
plt.figure(figsize=(12,6))
plt.title("CTR x Tempo por conjunto de anúncio")

a = sns.lineplot(x='date_start', y='ctr', data=df_filter_adsets, ci=None, hue='name')

In [ ]:
#ctr acumulado
plt.figure(figsize=(12,6))
plt.title("CTR acumulado x Tempo por conjunto de anúncio")

a = sns.lineplot(x='date_start', y='ctr_acc', data=df_filter_adsets, ci=None, hue='name')

#### 4.4.5. CPA

In [ ]:
plt.figure(figsize=(12,6))
plt.title("CPA x Tempo por conjunto de anúncio")

a = sns.lineplot(x='date_start', y='cpa', data=df_filter_adsets, ci=None, hue='name')

In [ ]:
#cpa acumulado
plt.figure(figsize=(12,6))
plt.title("CPA acumulado x Tempo por conjunto de anúncio")

a = sns.lineplot(x='date_start', y='cpa_acc', data=df_filter_adsets, ci=None, hue='name')

#### 4.4.6. ROAS

In [ ]:
plt.figure(figsize=(12,6))
plt.title("ROAS x Tempo por conjunto de anúncio")

a = sns.lineplot(x='date_start', y='purchase_roas', data=df_filter_adsets, ci=None, hue='name')

In [ ]:
#roas acumulado
plt.figure(figsize=(12,6))
plt.title("ROAS acumulado x Tempo por conjunto de anúncio")

a = sns.lineplot(x='date_start', y='purchase_roas_acc', data=df_filter_adsets, ci=None, hue='name')

#### 4.4.7. Video watched

In [ ]:
plt.figure(figsize=(12,6))
plt.title("Video-25% x Tempo por campanha")

a = sns.lineplot(x='date_start', y='video_25', data=df_filter_adsets, ci=None, hue='name')

In [ ]:
#video-25 acumulado
plt.figure(figsize=(12,6))
plt.title("Video-25% acumulado x Tempo por campanha")

a = sns.lineplot(x='date_start', y='video_25_acc', data=df_filter_adsets, ci=None, hue='name')

In [ ]:
plt.figure(figsize=(12,6))
plt.title("Video-50% x Tempo por campanha")

a = sns.lineplot(x='date_start', y='video_50', data=df_filter_adsets, ci=None, hue='name')

In [ ]:
#video-50 acumulado
plt.figure(figsize=(12,6))
plt.title("Video-50% acumulado x Tempo por campanha")

a = sns.lineplot(x='date_start', y='video_50_acc', data=df_filter_adsets, ci=None, hue='name')

In [ ]:
plt.figure(figsize=(12,6))
plt.title("Video-75% x Tempo por campanha")

a = sns.lineplot(x='date_start', y='video_75', data=df_filter_adsets, ci=None, hue='name')

In [ ]:
#video-75 acumulado
plt.figure(figsize=(12,6))
plt.title("Video-75% acumulado x Tempo por campanha")

a = sns.lineplot(x='date_start', y='video_75_acc', data=df_filter_adsets, ci=None, hue='name')

#### Indicadores por quantidade de anúncios

In [ ]:
plt.figure(figsize=(12,6))
plt.title("CPC acumulado x Tempo por qtde de anúncio")

a = sns.lineplot(x='date_start', y='cpc_acc', data=df_adsets_insights, ci=None, hue='ads_amount', style='ads_amount')

In [ ]:
plt.figure(figsize=(12,6))
plt.title("CPM acumulado x Tempo por qtde de anúncio")

a = sns.lineplot(x='date_start', y='cpm_acc', data=df_adsets_insights, ci=None, hue='ads_amount', style='ads_amount')

In [ ]:
plt.figure(figsize=(12,6))
plt.title("CTR acumulado x Tempo por qtde de anúncio")

a = sns.lineplot(x='date_start', y='ctr_acc', data=df_adsets_insights, ci=None, hue='ads_amount', style='ads_amount')

#### Correlacionando indicadores

In [ ]:
df_adsets_insights['spend_norm'] = df_adsets_insights['spend'] / np.amax(df_adsets_insights['spend'], axis=0)
df_adsets_insights['cpc_norm'] = df_adsets_insights['cpc'] / np.amax(df_adsets_insights['cpc'], axis=0)
df_adsets_insights['cpm_norm'] = df_adsets_insights['cpm'] / np.amax(df_adsets_insights['cpm'], axis=0)
df_adsets_insights['ctr_norm'] = df_adsets_insights['ctr'] / np.amax(df_adsets_insights['ctr'], axis=0)

In [ ]:
plt.figure(figsize=(12,6))
plt.title("KPI's x Tempo por campanha")

#a = sns.lineplot(x='date_start', y='spend_norm', data=df_filter, ci=None)
a = sns.lineplot(x='date_start', y='cpc_norm', data=df_filter_adsets, ci=None)
a = sns.lineplot(x='date_start', y='cpm_norm', data=df_filter_adsets, ci=None)
a = sns.lineplot(x='date_start', y='ctr_norm', data=df_filter_adsets, ci=None)

plt.legend(['cpc', 'cpm', 'ctr'])

## 5. coletando dados de anúncio

In [14]:
df_ads_full = pd.read_csv('ads_full.csv', sep=';')

In [15]:
df_ads_full['date_start'].min()

'2022-03-24'

In [16]:
df_ads = pd.DataFrame()

In [ ]:
#graph.get_ads_status(since_until=[(datetime(2022,11,24) - timedelta(days=0)).strftime("%Y-%m-%d"),(datetime(2022,11,24) - timedelta(days=0)).strftime("%Y-%m-%d")], increment=1, process=1, impressions=50000, effective_status=['CAMPAIGN_PAUSED'])

In [ ]:
for i in range(10):
    while True:
            try:
                df_temp = graph.get_ads_status(since_until=[(datetime(2022,3,13) - timedelta(days=i)).strftime("%Y-%m-%d"),(datetime(2022,3,13) - timedelta(days=i)).strftime("%Y-%m-%d")], increment=1, process=1, filter='GREATER_THAN', impressions=50000, effective_status=['CAMPAIGN_PAUSED'])
                #df_temp2 = graph.get_ads_status(since_until=[(datetime(2022,11,13) - timedelta(days=i)).strftime("%Y-%m-%d"),(datetime(2022,11,13) - timedelta(days=i)).strftime("%Y-%m-%d")], increment=1, process=1, filter='LESS_THAN_OR_EQUAL', impressions=50000, effective_status=['CAMPAIGN_PAUSED'])
                #df_temp3 = graph.get_ads_status(since_until=[(datetime(2022,11,13) - timedelta(days=i)).strftime("%Y-%m-%d"),(datetime(2022,11,13) - timedelta(days=i)).strftime("%Y-%m-%d")], increment=1, process=1, filter='GREATER_THAN', impressions=0, effective_status=['PAUSED','ACTIVE','ADSET_PAUSED'])
                df_ads = df_ads.append(df_temp)
                #df_ads = df_ads.append(df_temp2)
                #df_ads = df_ads.append(df_temp3)
                #if KeyError:
                #    raise Exception("Timeout")
                print(i)
                break
            except:
                time.sleep(30)
                print('erro',i)

In [ ]:
for i in range(10):
    while True:
            try:
                df_temp = graph.get_ads_status(since_until=[(datetime(2022,3,13) - timedelta(days=i)).strftime("%Y-%m-%d"),(datetime(2022,3,13) - timedelta(days=i)).strftime("%Y-%m-%d")], increment=1, process=1, filter='LESS_THAN_OR_EQUAL', impressions=50000, effective_status=['CAMPAIGN_PAUSED'])
                df_ads = df_ads.append(df_temp)
                print(i)
                break
            except:
                time.sleep(30)
                print('erro',i)

In [ ]:
for i in range(10):
    while True:
            try:
                df_temp = graph.get_ads_status(since_until=[(datetime(2022,3,13) - timedelta(days=i)).strftime("%Y-%m-%d"),(datetime(2022,3,13) - timedelta(days=i)).strftime("%Y-%m-%d")], increment=1, process=1, filter='GREATER_THAN', impressions=0, effective_status=['PAUSED','ACTIVE','ADSET_PAUSED'])
                df_ads = df_ads.append(df_temp)
                print(i)
                break
            except:
                time.sleep(30)
                print('erro',i)

In [ ]:
df_ads_data = df_ads[df_ads['insights'].notna()].reset_index(drop=True)

In [ ]:
df_ads_data_insights = pd.DataFrame()

In [ ]:
df4 = pd.DataFrame({'name':None}, index=[0])

#df2 = pd.DataFrame({'name':[None],'product':[None],'type':[None],'date_start':[None],'date_stop':[None],'spend':[None]})
for i in range(len(df_ads_data)):
    for j in range(len(df_ads_data['insights'][i]['data'])):
        df4['name'] = df_ads_data['name'][i]
        df4['status'] = df_ads_data['status'][i]
        df4['product'] = df_ads_data['product'][i]
        df4['kind'] = df_ads_data['kind'][i]
        df4['objective'] = df_ads_data['objective'][i]
        df4['adset_name'] = df_ads_data['adset_name'][i]
        df4['insta_link'] = df_ads_data['insta_link'][i]
        df4['thumb_link'] = df_ads_data['thumb_link'][i]
        df4['date_start'] = df_ads_data['insights'][i]['data'][j]['date_start']
        df4['date_stop'] = df_ads_data['insights'][i]['data'][j]['date_stop']
        df4['spend'] = float(df_ads_data['insights'][i]['data'][j]['spend'])
        df4['clicks'] = int(df_ads_data['insights'][i]['data'][j]['clicks'])
        df4['impressions'] = int(df_ads_data['insights'][i]['data'][j]['impressions'])
        df4['frequency'] = float(df_ads_data['insights'][i]['data'][j]['frequency'])
        df4['reach'] = int(df_ads_data['insights'][i]['data'][j]['reach'])
        df4['ctr'] = df_ads_data['insights'][i]['data'][j].get('ctr') if df_ads_data['insights'][i]['data'][j].get('ctr') is None else float(df_ads_data['insights'][i]['data'][j].get('ctr'))
        df4['cpc'] = df_ads_data['insights'][i]['data'][j].get('cpc') if df_ads_data['insights'][i]['data'][j].get('cpc') is None else float(df_ads_data['insights'][i]['data'][j].get('cpc'))
        df4['cpm'] = df_ads_data['insights'][i]['data'][j].get('cpm') if df_ads_data['insights'][i]['data'][j].get('cpm') is None else float(df_ads_data['insights'][i]['data'][j].get('cpm'))
        df4['video_25'] = df_ads_data['insights'][i]['data'][j].get('video_p25_watched_actions') if df_ads_data['insights'][i]['data'][j].get('video_p25_watched_actions') is None else int(df_ads_data['insights'][i]['data'][j]['video_p25_watched_actions'][0]['value'])
        df4['video_50'] = df_ads_data['insights'][i]['data'][j].get('video_p50_watched_actions') if df_ads_data['insights'][i]['data'][j].get('video_p50_watched_actions') is None else int(df_ads_data['insights'][i]['data'][j]['video_p50_watched_actions'][0]['value'])
        df4['video_75'] = df_ads_data['insights'][i]['data'][j].get('video_p75_watched_actions') if df_ads_data['insights'][i]['data'][j].get('video_p75_watched_actions') is None else int(df_ads_data['insights'][i]['data'][j]['video_p75_watched_actions'][0]['value'])
        
        try:
            x = df_ads_data['insights'][i]['data'][j]['actions'].index({'action_type': 'purchase', 'value': '1'})
            df4['purchase'] = int(df_ads_data['insights'][i]['data'][j]['actions'][x]['value'])
            df4['purchase_roas'] = float(df_ads_data['insights'][i]['data'][j]['purchase_roas'][0]['value'])
        except:
            df4['purchase'] = 0
            df4['purchase_roas'] = 0
            
        for k in range(len(df_ads_data['targetingsentencelines'][i]['targetingsentencelines'])):
            try:
                col = df_ads_data['targetingsentencelines'][i]['targetingsentencelines'][k]['content']
                row = df_ads_data['targetingsentencelines'][i]['targetingsentencelines'][k]['children']
                df4[col] = row
            except:
                df4[col] = 0
                
        
        df_ads_data_insights = df_ads_data_insights.append(df4)
        
df_ads_data_insights.reset_index(drop=True, inplace=True)

for i in range(len(df_ads_data_insights['Posicionamentos:'])):
    df_ads_data_insights['Posicionamentos:'][i] = df_ads_data_insights['Posicionamentos:'][i].split(',')

In [ ]:
df_ads_full = df_ads_full.append(df_ads_data_insights)
df_ads_full.reset_index(drop=True, inplace=True)
df_ads_full['date_start'] = pd.to_datetime(df_ads_full['date_start'])
df_ads_full['date_stop'] = pd.to_datetime(df_ads_full['date_stop'])

In [10]:
df_ads_full.to_csv('ads_full.csv', sep=';', index=False, decimal='.')

In [ ]:
pd.options.display.max_columns = 99
df_ads_full

In [ ]:
#df_ads_full.drop(columns=['purchase_acc','spend_acc','cpa','cpa_acc','purchase_value','purchase_value_acc','purchase_roas_acc','clicks_acc','impressions_acc','reach_acc','cpc_acc',
 #                         'cpm_acc', 'ctr_acc','video_25_acc','video_25_acc','video_25_acc'], axis=1, inplace=True)

In [ ]:
'''
#dados acumulados por campanha
df_ads_data_insights['purchase_acc'] = df_ads_data_insights.groupby('name')['purchase'].cumsum()
df_ads_data_insights['spend_acc'] = df_ads_data_insights.groupby('name')['spend'].cumsum()
df_ads_data_insights['cpa'] = df_ads_data_insights['spend'] / df_ads_data_insights['purchase']
df_ads_data_insights['cpa_acc'] = df_ads_data_insights['spend_acc'] / df_ads_data_insights['purchase_acc']

'''
'''
for i in range(len(df_ads_data_insights)):
    if df_ads_data_insights['purchase'][i] != 0:
        df_ads_data_insights['cpa'][i] = df_ads_data_insights['spend'][i] / df_ads_data_insights['purchase'][i]
    else:
        df_ads_data_insights['cpa'][i] = df_ads_data_insights['spend'][i]
        
for i in range(len(df_ads_data_insights)):
    if df_ads_data_insights['purchase_acc'][i] != 0:
        df_ads_data_insights['cpa_acc'][i] = df_ads_data_insights['spend_acc'][i] / df_ads_data_insights['purchase_acc'][i]
    else:
        df_ads_data_insights['cpa_acc'][i] = df_ads_data_insights['spend_acc'][i]
'''
'''
  
df_ads_data_insights['purchase_value'] = df_ads_data_insights['purchase_roas'] * df_ads_data_insights['spend']
df_ads_data_insights['purchase_value_acc'] = df_ads_data_insights.groupby('name')['purchase_value'].cumsum()
df_ads_data_insights['purchase_roas_acc'] = df_ads_data_insights['purchase_value_acc'] / df_ads_data_insights['spend_acc']

df_ads_data_insights['clicks_acc'] = df_ads_data_insights.groupby('name')['clicks'].cumsum()
df_ads_data_insights['impressions_acc'] = df_ads_data_insights.groupby('name')['impressions'].cumsum()
df_ads_data_insights['reach_acc'] = df_ads_data_insights.groupby('name')['reach'].cumsum()
df_ads_data_insights['cpc_acc'] = df_ads_data_insights['spend_acc'] / df_ads_data_insights['clicks_acc']
df_ads_data_insights['cpm_acc'] = (df_ads_data_insights['spend_acc'] / df_ads_data_insights['impressions_acc']) * 1000
df_ads_data_insights['ctr_acc'] = df_ads_data_insights['clicks_acc'] / df_ads_data_insights['impressions_acc'] * 100 #em %

df_ads_data_insights.replace([np.inf, -np.inf], 0, inplace=True)

df_ads_data_insights['video_25_acc'] = df_ads_data_insights.groupby('name')['video_25'].cumsum()
df_ads_data_insights['video_50_acc'] = df_ads_data_insights.groupby('name')['video_50'].cumsum()
df_ads_data_insights['video_75_acc'] = df_ads_data_insights.groupby('name')['video_75'].cumsum()
'''